In [19]:
import os
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

os.environ["OPEN_AI_KEY"] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")






In [20]:
import pandas as pd

data = pd.read_csv('../data/customers-100.csv')

data.head()

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/


In [21]:
# Todos:
# 1. loading and splitting the file
# 2. Vector store creation using FAISS and openai embeddings
# 3. Retriever setup for querying the processed document
# 4: creating a question and answer on the csv file

# 1. loading and splitting the file
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()


In [28]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
index = faiss.IndexFlatL2(len(OpenAIEmbeddings().embed_query(" ")))

vector_store = FAISS(
  embedding_function=OpenAIEmbeddings(),
  index=index,
  docstore=InMemoryDocstore(),
  index_to_docstore_id={}
)


In [29]:
vector_store.add_documents(documents=docs)

['c145afd3-c3ec-42f6-8540-e466996fb5e6',
 'd3de9dd2-3a2d-4261-bea5-ff29d4d0b998',
 '21ad0488-39ed-427a-b507-7abc33b2fac0',
 'a43e4cac-463a-4dfe-960b-4bbcc8a0e160',
 'e289fc6b-92ac-42e9-9c2c-f895eaf4125b',
 '23c16f6b-0d1b-47b4-95f5-22bb3cb9f45e',
 '02e4e472-8781-4477-8292-8601a775cfcb',
 'ed57d3c2-56a6-4d83-9681-88465e9d465a',
 '04500626-812f-49dd-af78-94181c0872e4',
 'c94e27bf-af83-4f03-bf24-6655aed13945',
 'b0673fe5-4302-43e0-a31b-be0bf28d2488',
 'c4802bd7-c44a-4dd7-a3b3-121554fa86ed',
 '116bd4b0-6b32-494d-b42a-261e3fc79ab1',
 '376b963a-2ce9-4ac7-9eb4-b5a11e5ae4fb',
 '2b0bc588-b537-4f7d-9411-f1de23fad50d',
 '04020466-1a9b-44c6-a6b7-eb81ac927b9e',
 'c9e76e6f-6f88-4bc2-85c8-902aae538bf2',
 'c3542fb3-c3f2-48dd-ab98-50cbbd3e12bc',
 '35c56db3-eaad-46bd-99d4-b4c1d60f7a6e',
 '27ce72d3-ffe1-43b6-8408-010ad0162586',
 '46a73eda-2324-4604-87bb-a66c69a9b73c',
 'd493d101-2ee4-4123-b709-e844122103f8',
 'a7574bda-87e0-4560-b312-6f98dd3659f4',
 '529468dd-089e-4ce3-b752-813c4434bf16',
 'f0a2d0b6-4a40-

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = vector_store.as_retriever()


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
  ("system", system_prompt),
  ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [ ]:
answer= rag_chain.invoke({"input": "which company does sheryl Baste work for?"})
answer['answer']